# (실습) 플로이드-워셜 알고리즘

**문제: 플로이드-워셜 알고리즘**

아래 `floyd_warshall()` 함수는 $n$개의 행렬 $D^{(0)}$, $D^{(1)}$, ..., $D^{(n)}$ 를
사용한다.

In [1]:
from copy import deepcopy

def floyd_warshall(W):
    n = len(W)
    # 사전을 이용하여 D^(0), ... , D^(n) 저장
    # 키는 0, 1, ..., n 사용
    D = dict() 

    # D^(0) 지정
    # 주의: deepcopy를 사용하지 않으면 W가 수정됨
    D[0] = deepcopy(W)

    # D^(k) 로부터 D^(k+1)를 생성
    for k in range(0, n):
        D[k+1] = D[k]
        # 행렬의 인덱스는 0부터 (n-1)까지 이동
        for i in range(0, n):
            for j in range(0, n):
                if D[k][i][k]+ D[k][k][j] < D[k][i][j]:
                    D[k+1][i][j] = D[k][i][k]+ D[k][k][j]
    
    # 최종 완성된 D[n] 반환
    return D[n]

위 알고리즘을 한 개의 행렬만 사용하도록 알고리즘을 수정하라.
이를 위해 아래 아이디어를 이용한다.

- `D = deepcopy(W)`로 초기화
- `k`에 대해 반복문을 실행 할 때, 새로운 `D`를 생성하는 대신 `D` 자체를 업데이트

견본 답안:

`D[k+1] = D[k]` 는 `D[k+1]`와 `D[k]`가 동일한 행렬을 가리키도록 한다.
따라서 하나의 행렬 `D = deepcopy(W)` 를 선언한 다음 `D` 를 계속 업데이트해도 동일한 결과를 얻는다.
이유는 `D[k+1][i][j] = D[k][i][k]+ D[k][k][j]`를 계산할 때 
`D[k+1][i][k]`와 `D[k+1][k][j]`는 미리 업데이트 되지 않기 때문이다.
미리 업데이트 되지 않는 이유는 `D[k+1][i][k] = D[k][i][k]`와 `D[k+1][k][j]=D[k][k][j]` 가 성립하기 때문이다.

In [1]:
from copy import deepcopy

def floyd_warshall(W):
    n = len(W)

    # D^(0) 지정
    # 주의: deepcopy를 사용하지 않으면 W에 혼란을 발생시킴
    D = deepcopy(W)

    # k가 0부터 (n-1)까지 이동하면서 D가 D^(1), ..., D^(n)을 차례대로 모방함.
    # 즉, D를 업데이트하는 방식을 이용하여 최종적으로 D^(n) 생성
    for k in range(0, n):
        # 행렬의 인덱스는 0부터 (n-1)까지 이동
        for i in range(0, n):
            for j in range(0, n):
                if D[i][k]+ D[k][j] < D[i][j]:
                    D[i][j] = D[i][k]+ D[k][j]
    
    # 최종 완성된 D 반환
    return D

In [2]:
# 무한에 해당하는 기호 사용
from math import inf

# inf 는 두 노드 사이에 간선이 없음을 의미함.
W = [[0, 1, inf, 1, 5],
     [9, 0, 3, 2, inf],
     [inf, inf, 0, 4, inf],
     [inf, inf, 2, 0, 3],
     [3, inf, inf, inf, 0]]

In [27]:
floyd_warshall(W)

NameError: name 'floyd_warshall' is not defined

**문제: 0-1 배낭채우기 문제**

W kg까지 넣을 수 있는 가방을 들고 쥬얼리샵에 침입하였다고 가정한다.
훔칠 수 있는 N 개의 보석이 주어졌고 각각이 서로 다른 무게를 갖는다고 가정한다.
이때 최대의 값어치가 되도록 가방에 보석을 넣는 방법을 알아내는 문제인
0-1 배낭채우기<font size='2'>0-1 Knapsack problem</font> 문제의 알고리즘을 동적계획법으로 구현하려 한다.

문제 이해를 위해 다음 경우를 가정한다. 

- W = 20
- 훔칠 수 있는 보석이 종류별로 1개, 즉 총 N=5개.

| 보석 종류| 무게 | 값어치 |
| :---: | :---: | :---: |
| 0 | 2 | 3 |
| 1 | 3 | 4 |
| 2 | 4 | 8 |
| 3 | 5 | 8 |
| 4 | 9 | 10 |

동적계획법을 적용하기 위해 다음 성질을 만족하는 W x N 모양의 2차원 행렬 M을 사용한다.

```
M[i][j]: 처음부터 i번 째까지의 물건을 살펴보고, 배낭의 용량이 j였을 때 배낭에 들어간 물건들의 최대 가치
```

보다 자세한 내용과 알고리즘 설명은 많은 인터넷 사이트를 참고할 수 있다. 
예를 들어 아래 두 링크를 추천한다.

- [Knapsack problem (Wikipedia)](https://en.wikipedia.org/wiki/Knapsack_problem)
- [0-1 Knapsack problem (GeeksforGeeks)](https://www.geeksforgeeks.org/0-1-knapsack-problem-dp-10/)

**질문 1**

위 알고리즘을 동적계획법으로 구현하라.

**질문 2**

앞서 설명한 동적계획법을 적용했을 때 최소 비용이 계산되는 것을 보장하기 위해 먼저 최적의 원칙이 보장됨을 설명하라.

**질문 1 모범답안**

In [25]:
def Knapsack(W, wight, price):
    n = len(weight)
    M = [[0 for _ in range(W + 1)] for _ in range(n + 1)]

    for i in range(n + 1):
        for w in range(W + 1):
            # 아무것도 넣지 않거나 가방의 용량이 0인 경우
            if i == 0 or w == 0:
                M[i][w] = 0
            # i 번째 보석의 무게가 가방 용량보다 같거나 적은 경우
            # 
            elif wight[i-1] <= w:
                M[i][w] = max(price[i-1] + M[i-1][w-wight[i-1]], M[i-1][w])
            # i 번째 보석의 무게가 가방 용량보다 큰 경우
            # i 번째 보석 포기
            else:
                M[i][w] = M[i-1][w]

    return M[n][W]

In [26]:
W = 20
weight = [2, 3, 4, 5, 9]
price = [3, 4, 8, 8, 10]

Knapsack(W, weight, price)

29

**질문 2 모범답안**

이 성질은 큰 문제의 최적해가 작은 부분 문제의 최적해를 포함하고 있음을 의미하는데, 전체 문제의 최적해를 부분 문제의 최적해로 나눌 수 있다. 0-1 배낭채우기 문제에서는 현재 상태에서 최선의 선택을 했을 때, 이 선택이 전체 문제의 최적해의 부분으로 사용될 수 있다. 이를 통해 작은 배낭의 최적해를 조합하여 큰 배낭의 최적해를 찾을 수 있다.

**문제: 편집 거리 문제**

str1과 str2 두 개의 문자열이 주어졌을 때 문자열 str1을 문자열 str2로 변환하는 데 필요한 최소 비용을 가리키는
편집 거리를 계산하는 문제이다.
일명 **레벤슈타인 거리**<font size='2'>Levenshtein distance</font> 문제라고도 불린다..
단, 변환은 다음 세 가지 방식 중에 하나를 연속적으로 선택해서 진행한다.

- 하나의 문자를 그대로 사용. 비용은 5.
- 하나의 문자를 삭제. 비용은 20.
- 하나의 문자를 추가. 비용은 20

예를 들어, "algorithm"에서 "alligator"로의 변환에 필요한 최소 비용은 다음과 같이 구할 수 있다.

- 처음 두 개의 문자 "al"은 동일하기 때문에 그대로 사용. 비용 10.
- "gorithm"에서 "ligator"로의 변환에 필요한 최소 비용 계산.

따라서 두 개의 문자열의 길이의 합이 적은 경우의 편집 거리를 이용하여 보다 긴 두 문자열의 편집 거리를 
동적계획법으로 계산할 수 있다.
보다 자세한 내용과 알고리즘 설명은 많은 인터넷 사이트를 참고할 수 있다. 
예를 들어 아래 두 링크를 추천한다.

- [Levenshtein distance(Wikipedia)](https://en.wikipedia.org/wiki/Levenshtein_distance)
- [Damerau-Levenshtein distance (GeeksforGeeks)](https://www.geeksforgeeks.org/damerau-levenshtein-distance/)

`len(str1) = m`,  `len(str2) = n` 일 때, 동적 계획법을 적용하기 위해 
`(m+1, n+1)` 모양의 2차원 행렬 `P`를 아래 성질을 만족하도록 업데이트 한다.

- `P[i][j]`: 첫째 문자열의 길이가 i, 둘째 문자열의 길이가 j일 때, 두 문자열의 편집 거리.

**질문 1**

두 문자열의 편집 거리를 계산하는 함수 `edit_distance(str1, str2)`를 동적계획법으로 구현하라.

**질문 2**

앞서 설명한 동적계획법을 적용했을 때 최소 비용이 계산되는 것을 보장하기 위해 먼저 최적의 원칙이 보장됨을 설명하라.

**질문 1 모범답안**

`P` 가 아래 점화식을 만족한다.

- `str2`의 길이가 0인 경우: `str1`에 포함된 모든 문자 삭제. 
        
        P[i][0] = i*20

- `str2`의 길이가 0인 경우: `str1`에 포함된 모든 문자 삭제. 

        P[0][j] = j*20

`str1`과 `str2` 모두 길이가 1 이상인 경우엔 두 가지 경우로 분류하여 `P[i][j]`를 계산한다.

- 첫째: `str1[i-1]` 과 `str2[j-1]` 이 동일한 문자인 경우엔 동일한 문자를 그대로 두고 이후의 두 부분 문자열을 편집 비용을 사용함.

        P[i-1][j-1] + 5

- 둘째:`str1[i-1]` 과 `str2[j-1]` 이 서로 다른 문자인 경우엔 `str1[i-1]`을 삭제하고 나머지를 `str2`와 비교하거나,
    `str2[j-1]`을 삭제하고 나머지를 `str1`과 비교하거나,
    `str1[i-1]`과 `str2[j-1]` 모두 동일한 문자로 대체하거나 등
    세 가지 경우를 계산한 다음에 최소 편집 비용 선택.
    
    - `str1[i-1]` 삭제: `P[i-1][j] + 20`
    - `str1[i-1]`에 `str2[j-2]` 삽입: `P[i][j-1] + 20`
    - `str1[i-1]`과 `str2[j-1]` 모두 동일한 다른 문자로 대체: `P[i-1][j-1] + 40`

In [21]:
def edit_distance(str1, str2):
    m = len(str1)
    n = len(str2)

    # P 초기화. 0으로 채움
    P = [[0 for x in range(n + 1)] for y in range(m + 1)]

    # str2 가 빈 문자열인 경우: str1의 모든 문자 삭제
    for i in range(1, m + 1):
        P[i][0] = 20 * i

    # str1 이 빈 문자열인 경우: str2에 포함된 모든 문자를 차례대로 str1에 추가
    for j in range(1, n + 1):
        P[0][j] = 20 * j       # 그대로 사용하는 비용 +20

    # 두 문자열 모두 빈 문자열이 아닌 경우
    # 비교 대상인 두 문자열의 두 부분문자열의 첫째 문자가 동일한지 여부에 따라 다르게 계산
    # str1[i-1] : 비교 문자열의 첫 문자 (str1의 부분 문자열)
    # str2[i-1] : 비교 문자열의 첫 문자 (str2의 부분 문자열)

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # 두 비교 문자열 각각의 첫 문자가 서로 같은 경우
            # 동일한 두 문자를 그대로 두고 다음 두 개의 문자부터 비교
            if str1[i - 1] == str2[j - 1]:
                P[i][j] = P[i-1][j-1] + 5
                
            # 두 비교 문자열 각각의 첫 문자가 서로 다른 경우
            # 세 가지 경우의 최소 비용 선택
            else:
                # str1[i-1] 삭제
                x_remove = P[i-1][j] + 20
                # str1[i-1]에 str2[j-1] 삽입
                x_insert = P[i][j-1] + 20
                # str1[i-1]과 str2[j-1] 모두 동일한 다른 문자로 대체
                x_replace = P[i-1][j-1] + 40
                
                # 세 경우의 최소값 선택
                P[i][j] = min(x_insert, x_remove, x_replace)

    # 행렬과 최종 비용 함께 반환
    return P, P[m][n]

In [22]:
str1 = 'algorithm'
str2 = 'alligator'

P, cost = edit_distance(str1, str2)

편집거리 행렬은 다음과 같다.

In [23]:
P

[[0, 20, 40, 60, 80, 100, 120, 140, 160, 180],
 [20, 5, 25, 45, 65, 85, 105, 125, 145, 165],
 [40, 25, 10, 30, 50, 70, 90, 110, 130, 150],
 [60, 45, 30, 50, 70, 55, 75, 95, 115, 135],
 [80, 65, 50, 70, 90, 75, 95, 115, 100, 120],
 [100, 85, 70, 90, 110, 95, 115, 135, 120, 105],
 [120, 105, 90, 110, 95, 115, 135, 155, 140, 125],
 [140, 125, 110, 130, 115, 135, 155, 140, 160, 145],
 [160, 145, 130, 150, 135, 155, 175, 160, 180, 165],
 [180, 165, 150, 170, 155, 175, 195, 180, 200, 185]]

편집 비용은 다음과 같다.

In [24]:
print("편집비용:", cost)

편집비용: 185


**질문 2 모범답안**

앞서 소개한 동적 계획법은 문자열을 문자열의 헤드와 꼬리로 쪼개는 분할을 사용한다.

- 헤드: 문자열의 첫째 문자
- 꼬리: 헤드를 제외한 나머지 문자열

그런데 두 문자열의 편집 거리는 두 문자열 각각의 헤드의 동일성 여부와
두 문자열 각각의 꼬리를 편집하는 비용에 의존한다.
따라서 주어진 두 문자열의 최소 편집 비용은 분할을 통해 발생하는 경우의 최소 편집 비용을 포함한다.
즉, 최적의 원칙이 성립한다.